# // Getting NFL Data
___
The first step in getting this project underway is going to be getting **massive** amounts of NFL data from the web. I will be working in this notebook to "show my work" and for others to learn how to if they're curious. Ultimately, I'll also turn it into a regular `.py` Python script that you can run if you're so inclined.

For that, we're going to rely on the `requests` and `BeautifulSoup` libraries to glean information from:

- [FantasyPros](https://www.fantasypros.com/)
- [Pro Football Reference](https://www.pro-football-reference.com/)
- [FFToday](http://www.fftoday.com/stats/)
- [The Football Database](https://www.footballdb.com/fantasy-football/index.html)

**Editor's Note:** After researching several for quite some time and attempting to use many of them, I will be going with **FantasyPros** for reasons outlined below.

In [1]:
# Importing our necessary libraries
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import requests
import re

from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
%matplotlib inline

## FantasyPros
I'm choosing to go with this FantasyPros for the following reasons:
- You don't have to register to access the data
- Separately, I am currently a user and like their site overall
- Has available .5 PPR scoring data (FanDuel does .5 PPR so DFS appeal, and it will be most applicable for most leagues)
- The Football Database seemed like it was missing quite a few players from each week
- FFToday scoring was inconsistent in many places for .5 PPR scoring

#### Following the FanDuel vein, since Daily Fantasy Sports (DFS) is where the biggest value is going to be coming from, I will be scraping data for the following positions and referring to them from here on out by the name in parentheses:
1. Quarterback (QB)
2. Running Back (RB)
3. Wide Receiver (WR)
4. Tight End (TE)
5. Defense / Special Teams (DST)


> - For a more detailed breakdown on positional scoring, please reference FanDuel's scoring and rules reference [here](https://www.fanduel.com/rules).
- For a quick reference on what each column means, check out ESPN's stat reference [here](http://www.espn.com/nfl/news/story?id=2128923).

### Scraping for our QBs

1. Let's make some empty lists to throw all of our data into:

In [2]:
player = []
pass_comp = []
pass_att = []
pass_pct = []
pass_yds = []
yds_per_att = []
pass_TD = []
pass_INT = []
sacks_taken = []
rush_att = []
rush_yds = []
rush_TD = []
fumbles_lost = []
active = []
fpoints = []
fpoints_g = []
own_pct = []

In [3]:
# Make sure to incoporate this into the scrape!
week = []
year = []

2. And a list of those lists to make `dataframe` creation nice and easy:

In [4]:
qb_stats_lists = [player, pass_comp, pass_att, pass_pct, pass_yds, 
                  yds_per_att, pass_TD, pass_INT, sacks_taken, 
                  rush_att, rush_yds, rush_TD, fumbles_lost, 
                  active, fpoints, fpoints_g, own_pct]



3. Doing the actual scrape by going through each week of the season (via each URL iteration), grabbing the right `table`, iterating through each `row` and then `cell`, and putting that data into the appropriate `lists`.

In [5]:
for week_number in range(1,6):

    res = requests.get('https://www.fantasypros.com/nfl/stats/qb.php?week={}&scoring=HALF&range=week'.format(week_number))
    soup = BeautifulSoup(res.content, 'lxml')

    for row in soup.find('div', {'class':'mobile-table'}).find('tbody').find_all('tr'):
        cells = row.find_all('td')
        for index, selection in enumerate(qb_stats_lists):
            selection.append(cells[index].text.lstrip().strip())
        # I want "week" to become a unique identifier to bring in opponents
        week.append(week_number)

4. Let's make an empty `dataframe` and then fill it using our list of lists:

In [6]:
qbdf = pd.DataFrame(columns= ['player', 'pass_comp', 'pass_att', 'pass_pct',
                              'pass_yds', 'yds_per_att', 'pass_TD', 'pass_INT', 
                              'sacks_taken', 'rush_att', 'rush_yds', 'rush_TD', 
                              'fumbles_lost', 'active', 'fpoints', 'fpoints_g', 'own_pct'])
qbdf

,player,pass_comp,pass_att,pass_pct,pass_yds,yds_per_att,pass_TD,pass_INT,sacks_taken,rush_att,rush_yds,rush_TD,fumbles_lost,active,fpoints,fpoints_g,own_pct


In [7]:
for index, column in enumerate(qbdf.columns):
    qbdf[column] = qb_stats_lists[index]

5. Let's add on our `week` and `year` information as well as a separate `team` column from our `player` column (to later be used to bring in opponent data).

In [8]:
# Week
qbdf['week'] = week
qbdf['week'] = qbdf.week.astype(str)

In [9]:
team = []
for individual in qbdf['player']:
    team.append(re.findall('\(([^\)]+)\)', individual)[0])
qbdf['team'] = team

6. And let's just clean up that `player` name a bit too with another `regex`:

In [10]:
qbdf['player'] = [ind[0] for ind in [re.findall('^.*?(?=\s\()', quarterback) for quarterback in qbdf['player']]]

7. While we're at the cleanup, we're going to need to do some with the `own_pct`:

In [11]:
qbdf['own_pct'] = qbdf.own_pct.apply(lambda x: x.strip('%'))
qbdf.own_pct.replace(to_replace='', value='0.0', inplace=True)

8. Since most if not all of these values were actually read in as `strings`, we'll need to convert them to numeric columns (with the exceptions of `player`, `team`, and `week`.

In [12]:
for x in [col for col in qbdf.columns if col not in ['player','week', 'team', 'year']]:
    qbdf[x] = qbdf[x].astype(float)
    

In [13]:
# Checking to see if we;ll be able to manipulate it how we want
qbdf.sort_values('fpoints', ascending=False).head(10)

,player,pass_comp,pass_att,pass_pct,pass_yds,yds_per_att,pass_TD,pass_INT,sacks_taken,rush_att,rush_yds,rush_TD,fumbles_lost,active,fpoints,fpoints_g,own_pct,week,team
460,Mitch Trubisky,19.0,26.0,73.1,354.0,13.6,6.0,0.0,1.0,3.0,53.0,0.0,0.0,1.0,43.5,43.5,25.3,4,CHI
30,Ryan Fitzpatrick,21.0,28.0,75.0,417.0,14.9,4.0,0.0,0.0,12.0,36.0,1.0,0.0,1.0,42.3,42.3,28.1,1,TB
262,Drew Brees,39.0,49.0,79.6,396.0,8.1,3.0,0.0,1.0,3.0,7.0,2.0,0.0,1.0,40.5,40.5,98.6,3,NO
272,Matt Ryan,26.0,35.0,74.3,374.0,10.7,5.0,0.0,3.0,4.0,12.0,0.0,0.0,1.0,40.2,40.2,94.3,3,ATL
447,Jared Goff,26.0,33.0,78.8,465.0,14.1,5.0,0.0,1.0,2.0,7.0,0.0,0.0,1.0,39.3,39.3,90.6,4,LAR
125,Ben Roethlisberger,39.0,60.0,65.0,452.0,7.5,3.0,0.0,1.0,2.0,9.0,1.0,0.0,1.0,39.0,39.0,95.1,2,PIT
217,Patrick Mahomes,23.0,28.0,82.1,326.0,11.6,6.0,0.0,1.0,5.0,18.0,0.0,0.0,1.0,38.8,38.8,98.7,2,KC
418,Andrew Luck,40.0,62.0,64.5,464.0,7.5,4.0,0.0,4.0,3.0,11.0,0.0,1.0,1.0,35.7,35.7,87.0,4,IND
177,Kirk Cousins,35.0,48.0,72.9,425.0,8.9,4.0,1.0,2.0,2.0,5.0,0.0,0.0,1.0,33.5,33.5,95.7,2,MIN
188,Blake Bortles,29.0,45.0,64.4,377.0,8.4,4.0,1.0,0.0,6.0,35.0,0.0,0.0,1.0,32.6,32.6,41.5,2,JAC


Through Week 5, no player has posted 2 of the season's Top 10 QB performances — yay parity!

In [20]:
qbdf.groupby('player')['fpoints'].mean().sort_values(ascending=False)[0:10]

player
Patrick Mahomes       26.22
Drew Brees            25.10
Matt Ryan             24.68
Jared Goff            22.96
Philip Rivers         22.36
Deshaun Watson        22.18
Kirk Cousins          21.94
Ben Roethlisberger    21.70
Aaron Rodgers         20.90
Andrew Luck           20.62
Name: fpoints, dtype: float64

Through Week 5, we see some familiar suspects in the Top 10 points per game, but also some young guns like Mahomes, Goff, Watson, Cousins, and a resurgence for Luck.

9. Now let's dump everything we have so far into a `csv` file that we can save to our current working directory:

In [14]:
qbdf.to_csv('qbstats_2018.csv')

### Doing the same thing for our RBs

In [22]:
player = []

rush_att = []
rush_yds = []
rush_ypc = []
long = []
over_20 = []
rush_TD = []

receptions = []
targets = []
rec_yds = []
rec_ypr = []
rec_TD = []

fumbles_lost = []
active = []
fpoints = []
fpoints_g = []
own_pct = []

week = []
year = []

In [23]:
rb_stats_lists = [player,rush_att,rush_yds,rush_ypc,long,over_20,rush_TD,
                  receptions,targets,rec_yds,rec_ypr,rec_TD,
                  fumbles_lost,active,fpoints,fpoints_g,own_pct]

for week_number in range(1,6):

    res = requests.get('https://www.fantasypros.com/nfl/stats/rb.php?week={}&scoring=HALF&range=week'.format(week_number))
    soup = BeautifulSoup(res.content, 'lxml')

    for row in soup.find('div', {'class':'mobile-table'}).find('tbody').find_all('tr'):
        cells = row.find_all('td')
        for index, selection in enumerate(rb_stats_lists):
            selection.append(cells[index].text.lstrip().strip())
        # I want "week" to become a unique identifier to bring in opponents
        week.append(week_number)

rbdf = pd.DataFrame(columns= ['player','rush_att','rush_yds','rush_ypc','long','over_20','rush_TD',
                  'receptions','targets','rec_yds','rec_ypr','rec_TD',
                  'fumbles_lost','active','fpoints','fpoints_g','own_pct'])

for index, column in enumerate(rbdf.columns):
    rbdf[column] = rb_stats_lists[index]

rbdf['week'] = week
rbdf['week'] = rbdf.week.astype(str)
    
team = []
for individual in rbdf['player']:
    team.append(re.findall('\(([^\)]+)\)', individual)[0])
rbdf['team'] = team
    
rbdf['player'] = [ind[0] for ind in [re.findall('^.*?(?=\s\()', quarterback) for quarterback in rbdf['player']]]    
    
rbdf['own_pct'] = rbdf.own_pct.apply(lambda x: x.strip('%'))
rbdf.own_pct.replace(to_replace='', value='0.0', inplace=True)    

for x in [col for col in rbdf.columns if col not in ['player','week', 'team', 'year']]:
    rbdf[x] = rbdf[x].astype(float)





In [29]:
rbdf.loc[rbdf['fpoints'] == rbdf.fpoints.ma]

,player,rush_att,rush_yds,rush_ypc,long,over_20,rush_TD,receptions,targets,rec_yds,rec_ypr,rec_TD,fumbles_lost,active,fpoints,fpoints_g,own_pct,week,team
189,Alvin Kamara,8.0,29.0,3.6,10.0,0.0,2.0,9.0,12.0,112.0,12.4,1.0,0.0,1.0,38.6,38.6,100.0,1,NO
978,Alvin Kamara,19.0,134.0,7.1,49.0,1.0,3.0,5.0,9.0,47.0,9.4,0.0,0.0,1.0,38.6,38.6,100.0,4,NO
1248,James Conner,21.0,110.0,5.2,30.0,1.0,2.0,4.0,4.0,75.0,18.8,0.0,0.0,1.0,32.5,32.5,98.2,5,PIT
941,Ezekiel Elliott,25.0,152.0,6.1,41.0,1.0,0.0,4.0,4.0,88.0,22.0,1.0,0.0,1.0,32.0,32.0,100.0,4,DAL
196,James Conner,31.0,135.0,4.4,22.0,1.0,2.0,5.0,6.0,57.0,11.4,0.0,1.0,1.0,31.7,31.7,98.2,1,PIT
1183,Todd Gurley,22.0,77.0,3.5,16.0,0.0,3.0,4.0,5.0,36.0,9.0,0.0,0.0,1.0,31.3,31.3,100.0,5,LAR
395,Todd Gurley,19.0,42.0,2.2,11.0,0.0,3.0,3.0,3.0,31.0,10.3,0.0,0.0,1.0,30.8,30.8,100.0,2,LAR
1168,Isaiah Crowell,15.0,219.0,14.6,77.0,3.0,1.0,1.0,1.0,12.0,12.0,0.0,0.0,1.0,29.6,29.6,79.7,5,NYJ
394,Melvin Gordon,9.0,28.0,3.1,20.0,1.0,1.0,6.0,7.0,38.0,6.3,2.0,0.0,1.0,27.6,27.6,100.0,2,LAC
921,Melvin Gordon,15.0,104.0,6.9,34.0,1.0,0.0,7.0,10.0,55.0,7.9,1.0,0.0,1.0,27.4,27.4,100.0,4,LAC


Stats to bring in:
- Matchups (including game location)
- Defensive stats
- Team play-calling
- Depth charts?
- Weather